In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 导入模块"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from selenium import webdriver\n",
    "import os\n",
    "import time\n",
    "import re\n",
    "import requests"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 创建数据库"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "from pony.orm import *\n",
    "\n",
    "\n",
    "db = Database()\n",
    "\n",
    "\n",
    "class Star(db.Entity):\n",
    "    id = PrimaryKey(int, auto=True)\n",
    "    name = Optional(str, column='name')\n",
    "    picture=Optional(str, column='picture')\n",
    "    href1=Optional(str, column='href1')\n",
    "    gender = Optional(str, column='gender')\n",
    "    href = Optional(str, column='href')\n",
    "    year = Optional(int, column='year')\n",
    "    month = Optional(int, column='month')\n",
    "    day = Optional(int, column='day')\n",
    "    xz = Optional(str, column='xz')\n",
    "    html = Optional(str, column='html')\n",
    "    address = Optional(str, column='address')\n",
    "    height = Optional(float, column='height')\n",
    "\n",
    "\n",
    "\n",
    "# db.generate_mapping()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "dbpath = 'f:/ex7/star.sqlite'\n",
    "if os.path.exists(dbpath):\n",
    "    os.remove(dbpath)\n",
    "f = open(dbpath,\"w\")\n",
    "f.close()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "db.bind(provider='sqlite',filename='f:/ex7/star.sqlite')\n",
    "\n",
    "db.generate_mapping(create_tables=True)\n",
    "\n",
    "set_sql_debug(True)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义将名字、链接、小图片，入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_data(h):\n",
    "    reobj = re.compile(r'<div class=\"op_exactqa_item c-gap-bottom c-span4 [\\d\\D]*?\"><div class=\"op_exactqa_feedback OP_LOG_BTN\">[\\d\\D]*?<a href=\"([\\d\\D]*?)\" title=\"([\\d\\D]*?)\" target=\"_blank\"><img class=\"[\\d\\D]*?\" src=\"([\\d\\D]*?)\">')\n",
    "    for match in reobj.finditer(h):\n",
    "        href2=match.group(1).strip()\n",
    "        name1=match.group(2).strip()\n",
    "        picture1=match.group(3).strip()\n",
    "        s=Star(name=name1,href1=href2,picture=picture1)\n",
    "        db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星百科链接并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_href(n):\n",
    "    star.href=\"https://baike.baidu.com/item/\"+n\n",
    "    time.sleep(2)\n",
    "    db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "#  定义获取所有详情页链接的源代码并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_html(h):\n",
    "    head = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36'}\n",
    "    r = requests.get(h,headers = head)\n",
    "    r.encoding = 'utf-8'\n",
    "    star.html = r.text\n",
    "    db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星的星座并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_xz(h):\n",
    "    if re.search(r'座</dt>\\n<dd class=\"basicInfo-item value\">\\n<a[\\d\\D]*?\">([\\d\\D]*?)</a',h):\n",
    "        match = re.search(r'座</dt>\\n<dd class=\"basicInfo-item value\">\\n<a[\\d\\D]*?\">([\\d\\D]*?)</a',h)\n",
    "        xz1 = match.group(1).strip()\n",
    "        star.xz = xz1\n",
    "        time.sleep(2)\n",
    "        db.commit()\n",
    "    elif re.search(r'座</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)\\n<',h):\n",
    "        match = re.search(r'座</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)\\n<',h)\n",
    "        xz1 = match.group(1).strip()\n",
    "        star.xz = xz1\n",
    "        time.sleep(2)\n",
    "        db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星的身高并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_height(h):\n",
    "    if re.search(r'高</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)\\n<',h):\n",
    "        height = re.search(r'高</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)\\n<',h).group(1).strip()\n",
    "        try:\n",
    "            if (\"cm\" in height) or (\"CM\" in height) or (\"Cm\" in height) or (\"厘米\" in height):\n",
    "                height = float(height[:-2])\n",
    "            elif  (\"m\" in height) or (\"M\" in height) or (\"米\" in height):\n",
    "                height = float(height[:-1])*100\n",
    "            star.height=height\n",
    "            time.sleep(2)\n",
    "            db.commit()\n",
    "        except:\n",
    "            pass\n",
    "    elif re.search(r'高</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)<sup',h):\n",
    "        height = re.search(r'高</dt>\\n<dd class=\"basicInfo-item value\">\\n([\\d\\D]*?)<sup',h).group(1).strip(\"/n\")\n",
    "        try:\n",
    "            if (\"cm\" in height) or (\"CM\" in height) or (\"Cm\" in height) or (\"厘米\" in height):\n",
    "                height = float(height[:-2])\n",
    "            elif  (\"m\" in height) or (\"M\" in height) or (\"米\" in height):\n",
    "                height = float(height[:-1])*100\n",
    "            star.height = height\n",
    "            time.sleep(2)\n",
    "            db.commit()\n",
    "        except:\n",
    "            pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星的性别并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_gender():\n",
    "    star.gender=\"女\"\n",
    "    time.sleep(2)\n",
    "    db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星的籍贯并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_address(h):\n",
    "    if re.search(r'<dt class=\"basicInfo-item name\">出生地[\\d\\D]*?\">([\\d\\D]*?)<',h):\n",
    "        match = re.search(r'<dt class=\"basicInfo-item name\">出生地[\\d\\D]*?\">([\\d\\D]*?)<',h)\n",
    "        jg1=match.group(1).strip()\n",
    "        star.address=jg1\n",
    "        time.sleep(2)\n",
    "        db.commit()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 定义获取明星的出生年，月，日并入库的函数"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_birth(h):\n",
    "    if  re.search(r'<dt class=\"basicInfo-item name\">出生日期[\\d\\D]*?\">([\\d\\D]*?)</dd>',h):\n",
    "        birth = re.search(r'<dt class=\"basicInfo-item name\">出生日期[\\d\\D]*?\">([\\d\\D]*?)</dd>',h).group(1).strip()\n",
    "        if \"年\" in birth and \"月\" in birth and \"日\" in birth: \n",
    "            year,month,day = re.split(\"[年月日]\",birth)[:3]\n",
    "            try:\n",
    "                star.year = int(year)\n",
    "                star.month = int(month)\n",
    "                star.day = int(day)\n",
    "                db.commit\n",
    "            except:\n",
    "                pass\n",
    "        elif \"年\" in birth and \"月\" in birth:\n",
    "            year,month = re.split(\"[年月]\",birth)[:2]\n",
    "            try:\n",
    "                star.year = int(year)\n",
    "                star.month = int(month)\n",
    "                db.commit\n",
    "            except:\n",
    "                pass\n",
    "        elif \"年\" in birth:\n",
    "            year = re.split(\"[年]\",bitrh)[:1]\n",
    "            try:\n",
    "                star.year = int(year)\n",
    "                db.commit\n",
    "            except:\n",
    "                pass\n",
    "        elif \"月\" in birth and \"日\" in birth: \n",
    "            month,day = re.split(\"[月日]\",birth)[:2]\n",
    "            try:\n",
    "                star.month = int(month)\n",
    "                star.day = int(day)\n",
    "                db.commit\n",
    "            except:\n",
    "                pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# 整体步骤"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "driver=webdriver.Chrome()\n",
    "url = 'https://www.baidu.com/s?ie=utf-8&f=8&rsv_bp=1&rsv_idx=2&tn=baiduhome_pg&wd=%E6%98%8E%E6%98%9F&rsv_spt=1&oq=%25E6%2598%258E%25E6%2598%259F&rsv_pq=a1a2cf770002ec2f&rsv_t=0e3baM8Xu8qPUVhj0snvRVkEdX1Le%2Bm%2FZvXgmFXh1EnnOrdvEFvLRImf6oviVMQdrqmk&rqlang=cn&rsv_enter=0&rsv_dl=tb'\n",
    "driver.get(url)\n",
    "time.sleep(2)\n",
    "driver.find_element_by_xpath(\"//div[@id='1']/div/div/div/div[2]/p/span[4]\").click()\n",
    "time.sleep(2)\n",
    "driver.find_element_by_xpath(\"//div[@id='1']/div/div/div/div[2]/p[2]/span[3]\").click()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "while True:\n",
    "    html = driver.page_source\n",
    "    get_data(html)\n",
    "    with db_session:\n",
    "        for star in Star.select():\n",
    "            get_href(star.name)\n",
    "            get_html(star.href)\n",
    "            get_xz(star.html)\n",
    "            get_height(star.html)\n",
    "            get_gender()\n",
    "            get_address(star.html)\n",
    "            get_birth(star.html)\n",
    "    match = re.search(r'<span class=\"opui-page-next OP_LOG_BTN\" style=\"([\\d\\D]*?)\">[\\d\\D]*?</span>', html)\n",
    "    if match.group(1)!=\"display: none;\":\n",
    "         driver.find_element_by_xpath(\"//div[@id='1']/div/div/div[2]/div[2]/p/span[6]\").click()\n",
    "    else:\n",
    "        break"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}